# WLS Regression & Wald Tests with Python
## Based on the work of Xavier Giné and Ghazala Mansuri

This Jupyter notebook goes through a similar regression process as that done by Giné and Mansuri in their work on the voting behavior of rural Pakistani women (see https://www.aeaweb.org/articles?id=10.1257/app.20130480 for the homepage of the research paper). This notebook reproduces most of the first column of Table 4, namely the WLS regression and the two Wald tests on the main covariates of interest. The specific .do file reproduced here is 'Table_4_OA11_OA16_Impact.do'.

### Part I: Import Statements and Juypter Notebook Options

In [1]:
import pandas as pd
import numpy as np
import statsmodels # just to get the version number
import statsmodels.formula.api as sm

In [2]:
# print version information for reference
print(pd.__version__)
print(np.__version__)
print(statsmodels.__version__)

0.22.0
1.14.0
0.8.0


In [3]:
# increase max columns displayed in Jupyter
pd.options.display.max_columns = 999

### Part II: Reading and Cleaning the Data

In [4]:
# read in the main .dta file
data = pd.io.stata.read_stata('/Users/adouglas/Downloads/6011/Data/Female_Voting_Lasso.dta')

In [5]:
# read in the .dta file containing the relevant weights
weights = pd.io.stata.read_stata('/Users/adouglas/Downloads/6011/Data/Prob_weights_bound_cluster.dta')

In [6]:
# merge the weight data with the main dataframe
# ignore the resulting error, has no impact on the data
data = data.merge(weights, how='left', on='neighborhood_code', validate='m:1')

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'double'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'double'


In [7]:
# drop the same columns as dropped by the researchers in their Stata .do file
data.drop(['village_name_y', 'neighborhoodtype', 'theory_prob', 'bound_2'], axis=1, inplace=True)

In [8]:
# rename the appropriate columns, in line with how researchers renamed columns
data.rename(index=str, columns={'village_name_x': 'village_name', 'actual_prob': 'prob_t1'}, inplace=True)

In [9]:
# insert a new column of data, which is dummy variable equal to 1 if household size was available for a given observation
data.insert(list(data.columns).index('hhsize')+1, 'd_hhsize', [1 if observ else 0 for observ in pd.notnull(data.hhsize)])

In [10]:
# replace missing hhsize values with 0
data['hhsize'] = data.apply(lambda row: 0 if np.isnan(row.hhsize) else row.hhsize, axis=1)

In [11]:
# check out the data before selecting regression columns
data.head(3)

,district_code,district_name,tehsil_code,tehsil_name,uc_code,uc_name,village_code,village_name,settlement_code,settlement_name,neighborhood_code,neighborhood_name,neighborhood_type,type_visit,censusid,women_number,women_name,hhh_name,hhh_father_name,vote_survey,NAregist_vot_F,zaat_status_low,dist_poll_HH,ngo_allow,sample_women,voted,vote_verified,check_ink,v_noinknovote,v_list,t1,t2,t,t1_m,t2_m,t_m,c_t1m,c_t2m,t_nocc,cc,t1_nocc,t2_nocc,cc1,cc2,age,age2,d_age,education,any_school,d_any_school,num_women,num_female,d_married,nc05,d_num_child0to5,zsh,nall,land,d_lown_total,month_expend,hhsize,d_hhsize,house_quality,asset_index,asset_index_sd,member_mrdo_cbo,voted_lastyear,dist_pollstat_sd,radio_access,tv_access,cable_access,advice_pir,vote_PPPP_F,mobil_ins_allow,voter_list,mobil_outs_allow,vote_past,party_voted_same,party_voted_dif,fem_voted_male_no,fem_no_voted,visit_receive,opinionindex1,fem_knowledgeindex,aft5feb_votelist,free_natelect,electday_violence,treat_200,treat_200_400,treat_400_600,treat_600_800,treat_800_1000,treat_1000_1200,sum_200,sum_200_400,sum_400_600,sum_600_800,sum_800_1000,sum_1000_1200,prob_t1,weight_t1,bound_1
0,1.0,Sukkur,1.0,Rohri,1.0,Trimoonh,1.0,Akbar Pur,1.0,Bachal Bhanbhro,5,Bachal Main Abadi,Treatment 2 (Module on Importance of Vote and ...,Treatment 2 (Module on Importance of Vote and ...,1.110101e+10,1,NAZEERAN,WAZEER,ALLAH RAKHO,1.0,346.0,1.0,0.0,no,yes,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,0.0,1.0,NaN,NaN,45.0,2025.0,0.0,No school,0.0,0.0,2.0,2.0,1.0,1.0,0.0,0.0,1.0,3.01250,0.0,9000.0,13.0,1,-1.563194,-1.562093,1.759311,yes,yes,1.997115,yes,yes,no,no,1.0,2.0,no,2.000000,1.0,1.0,0.0,0.0,0.0,0.0,1.033358,0.642816,yes,yes,no,10,9,15,9,5,12,14,12,17,10,5,15,0.319444,1.469388,No
1,1.0,Sukkur,1.0,Rohri,1.0,Trimoonh,1.0,Akbar Pur,1.0,Bachal Bhanbhro,5,Bachal Main Abadi,Treatment 2 (Module on Importance of Vote and ...,Treatment 2 (Module on Importance of Vote and ...,1.110101e+10,2,WAZEERAN,WAZEER,ALLAH RAKHO,1.0,346.0,1.0,0.0,yes,yes,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,0.0,1.0,NaN,NaN,25.0,625.0,0.0,No school,0.0,0.0,2.0,2.0,1.0,1.0,0.0,0.0,1.0,3.01250,0.0,9000.0,13.0,1,-1.563194,-1.562093,1.759311,no,no,1.997115,yes,yes,no,no,1.0,2.0,no,1.333333,0.0,1.0,0.0,0.0,0.0,0.0,-0.505169,-0.819066,yes,yes,no,10,9,15,9,5,12,14,12,17,10,5,15,0.319444,1.469388,No
2,1.0,Sukkur,1.0,Rohri,1.0,Trimoonh,1.0,Akbar Pur,1.0,Bachal Bhanbhro,5,Bachal Main Abadi,Treatment 2 (Module on Importance of Vote and ...,Treatment 2 (Module on Importance of Vote and ...,1.110101e+10,1,MAJEEDAN,MUKHTIAR,MUHAMMAD AALIM,0.0,346.0,1.0,0.0,yes,yes,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,0.0,1.0,NaN,NaN,45.0,2025.0,0.0,No school,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.50625,0.0,5000.0,8.0,1,-1.563194,-1.766873,1.759311,no,no,1.997115,yes,yes,yes,yes,NaN,2.0,yes,3.000000,0.0,0.0,0.0,0.0,1.0,0.0,1.033358,0.642816,no,yes,no,11,13,7,13,2,15,15,16,9,14,2,18,0.319444,1.469388,No


### Part III: Run Initial Regression

In [12]:
# select only the columns used as part of the regression analysis.
# this makes it easier for when we drop any observations with missing data, as doing so on the entire dataset would drop
# inappropriate observations from the dataset
reg_data = data[['v_noinknovote', 't1_nocc', 't2_nocc', 'NAregist_vot_F', 'hhsize',
                 'age', 'nc05', 'nall', 'tv_access', 'mobil_ins_allow', 'advice_pir',
                 'd_hhsize', 'd_age', 'd_num_child0to5', 'village_code', 'weight_t1', 'neighborhood_code']].copy()

In [13]:
# drop rows with missing data
reg_data.dropna(how='any', inplace=True)

In [14]:
# convert village_code values from floats to ints.
# this is important when creating dummy columns from the data, so that the column namess don't include decimals,
# i.e. village_code_2 vs. village_code_2.0
reg_data['village_code'] = reg_data['village_code'].astype(np.uint8)

In [15]:
# in line with our researchers, create dummy variables from the village_code column
reg_data = pd.get_dummies(reg_data, columns=['village_code'], drop_first=True)

In [16]:
# run our initial WLS regression
reg = sm.wls('v_noinknovote ~ t1_nocc + t2_nocc + NAregist_vot_F + hhsize + age + nc05 + nall + \
                              tv_access + mobil_ins_allow +  advice_pir + d_hhsize + d_age + \
                              d_num_child0to5 + village_code_2 + village_code_3 + village_code_4 + village_code_5 + \
                              village_code_6 + village_code_9 + village_code_10 + village_code_11',
                     data=reg_data, weights=reg_data['weight_t1'])

In [17]:
# fit our model with the appropriate options and get our results
results = reg.fit(use_t=True, cov_type='cluster', cov_kwds={'groups': reg_data['neighborhood_code']})

/Users/adouglas/Applications/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [18]:
# how does it look?
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          v_noinknovote   R-squared:                       0.198
Model:                            WLS   Adj. R-squared:                  0.191
Method:                 Least Squares   F-statistic:                     28.10
Date:                Thu, 18 Jan 2018   Prob (F-statistic):           4.30e-25
Time:                        09:24:42   Log-Likelihood:                -1422.2
No. Observations:                2304   AIC:                             2886.
Df Residuals:                    2283   BIC:                             3007.
Df Model:                          20                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1569      0.161      0.976      0.333      -0.164       0.478
tv_access[T.yes]      0.0701      0.030      2.323      0.023       0.010       0.130
advice_pir[T.yes]    -0.0909      0.026     -3.459      0.001      -0.143      -0.038
t1_nocc               0.0877      0.069      1.273      0.208      -0.050       0.225
t2_nocc               0.1252      0.070      1.780      0.080      -0.015       0.266
NAregist_vot_F        0.0006      0.000      4.699      0.000       0.000       0.001
hhsize               -0.0006      0.002     -0.347      0.730      -0.004       0.003
age                   0.0025      0.001      3.305      0.002       0.001       0.004
nc05                 -0.0095      0.009     -1.029      0.307      -0.028       0.009
nall                  0.1983      0.045      4.406      0.000       0.108       0.288
mobil_ins_allow       0.0611      0.019      3.201      0.002       0.023       0.099
d_hhsize              0.0054      0.101      0.053      0.958      -0.196       0.207
d_age                 0.0480      0.055      0.875      0.385      -0.062       0.157
d_num_child0to5       0.0480      0.055      0.875      0.385      -0.062       0.157
village_code_2       -0.3473      0.113     -3.067      0.003      -0.573      -0.121
village_code_3       -0.5972      0.124     -4.825      0.000      -0.844      -0.350
village_code_4       -0.3129      0.131     -2.392      0.020      -0.574      -0.052
village_code_5       -0.4217      0.150     -2.815      0.006      -0.721      -0.123
village_code_6        0.0504      0.117      0.432      0.667      -0.183       0.283
village_code_9       -0.3897      0.156     -2.498      0.015      -0.701      -0.078
village_code_10      -0.2229      0.140     -1.591      0.116      -0.503       0.057
village_code_11      -0.3092      0.125     -2.478      0.016      -0.558      -0.060
==============================================================================
Omnibus:                      419.327   Durbin-Watson:                   1.605
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              138.241
Skew:                          -0.376   Prob(JB):                     9.58e-31
Kurtosis:                       2.065   Cond. No.                     1.61e+20
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The smallest eigenvalue is 4.63e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### Part IV: Run Second Regression
As you can see from examining the summary above, d_age and d_num_child0to5 have the same exact values, indicating collinearity between these two variables. Stata automatically eliminates d_num_child0to5 when it runs the regression but statsmodels is not that smart.

This collinearity is a problem because the regression does not know what part of the variance in the data to ascribe to d_age and what part to ascribe to d_num_child0to5. If one wanted to look for this type of issue before running the regression, they could examine the correlation matrix of the pandas dataframe, like below.

In [19]:
# generate our correlation matrix and look at the results. Looking for any values higher than say 0.50
corr = reg_data.corr()
corr

,v_noinknovote,t1_nocc,t2_nocc,NAregist_vot_F,hhsize,age,nc05,nall,mobil_ins_allow,d_hhsize,d_age,d_num_child0to5,weight_t1,village_code_2,village_code_3,village_code_4,village_code_5,village_code_6,village_code_9,village_code_10,village_code_11
v_noinknovote,1.000000,-0.010511,0.058157,0.082750,-0.040898,0.139229,-0.071493,0.200342,0.083494,-0.023375,0.005094,0.005094,0.193443,-0.059574,-0.117095,0.001378,-0.024408,0.229013,-0.097422,-0.032042,0.023583
t1_nocc,-0.010511,1.000000,-0.679689,0.057337,0.023755,-0.019782,0.044300,0.005613,0.036238,-0.026783,0.000222,0.000222,0.592888,-0.059081,0.018288,0.027192,0.022953,0.017095,-0.011104,0.011681,0.017725
t2_nocc,0.058157,-0.679689,1.000000,-0.150008,-0.000519,-0.009345,0.001364,0.032952,-0.028206,0.006401,0.005215,0.005215,-0.381253,0.059819,-0.039921,-0.013954,-0.046353,-0.021905,0.055721,0.021802,-0.031811
NAregist_vot_F,0.082750,0.057337,-0.150008,1.000000,0.006172,0.000932,-0.015290,0.054313,0.024321,0.001778,0.007735,0.007735,-0.093963,-0.148884,0.327481,0.127241,0.272304,-0.045563,-0.032768,-0.293631,0.018244
hhsize,-0.040898,0.023755,-0.000519,0.006172,1.000000,-0.073893,0.116809,-0.018026,0.070379,0.330547,0.107881,0.107881,-0.040168,-0.038178,-0.003275,-0.140696,0.012006,-0.033919,0.237456,-0.006058,-0.004546
age,0.139229,-0.019782,-0.009345,0.000932,-0.073893,1.000000,-0.230823,0.345790,0.005313,0.007264,-0.177094,-0.177094,-0.017031,0.024544,0.040558,0.038909,0.006912,-0.009121,-0.063533,0.005067,-0.044856
nc05,-0.071493,0.044300,0.001364,-0.015290,0.116809,-0.230823,1.000000,0.016896,0.056422,-0.007116,-0.054469,-0.054469,-0.014119,-0.011458,0.042579,-0.033056,-0.063613,-0.057471,0.167251,-0.007520,-0.029967
nall,0.200342,0.005613,0.032952,0.054313,-0.018026,0.345790,0.016896,1.000000,0.063787,-0.039738,-0.014687,-0.014687,0.007324,-0.009002,-0.010599,0.013018,0.043205,-0.025300,0.091750,-0.021640,-0.009002
mobil_ins_allow,0.083494,0.036238,-0.028206,0.024321,0.070379,0.005313,0.056422,0.063787,1.000000,0.009434,0.037735,0.037735,0.028642,0.044945,0.084285,-0.142903,-0.100766,0.066591,0.044066,-0.066968,0.168285
d_hhsize,-0.023375,-0.026783,0.006401,0.001778,0.330547,0.007264,-0.007116,-0.039738,0.009434,1.000000,0.012630,0.012630,-0.020024,0.051675,-0.005300,-0.117983,-0.023115,0.004209,0.065493,0.011649,-0.023774


In [20]:
# given the large number of variables, it's a bit hard to examine the matrix by eye,
# so let's write a ridiculous for loop to spit out any large correlations
for row in corr.iterrows():
    for value in row[1].values:
        if abs(value) > 0.50:
            if row[1][row[1] == value].index[0] != row[1].name:
                print("{:<12} {:18} {:18}".format('{0: .3f}'.format(value), row[1][row[1] == value].index[0], row[1].name))

-0.680       t2_nocc            t1_nocc           
 0.593       weight_t1          t1_nocc           
-0.680       t1_nocc            t2_nocc           
 1.000       d_age              d_num_child0to5   
 1.000       d_age              d_num_child0to5   
 0.593       t1_nocc            weight_t1         


As you can see from the above, d_age and d_num_child0to5 are perfectly correlated. That's because they are dummy variables indicating the presense of two particular demographic variables, and in our case, if an observation is missing the age of the voter it's also missing the number of children aged 0 to 5 in the household.

Thus these two variables are exactly the same and one should be removed from the regression equation in order to improve the result.

In [21]:
# same regression as before, only with d_num_child0to5 removed
reg2 = sm.wls('v_noinknovote ~ t1_nocc + t2_nocc + NAregist_vot_F + hhsize + age + nc05 + nall + \
                                      tv_access + mobil_ins_allow +  advice_pir + d_hhsize + d_age + \
                                      village_code_2 + village_code_3 + village_code_4 + village_code_5 + \
                                      village_code_6 + village_code_9 + village_code_10 + village_code_11',
                     data=reg_data, weights=reg_data['weight_t1'])

In [22]:
# fit our model with the appropriate options and get our results
results2 = reg2.fit(use_t=True, cov_type='cluster', cov_kwds={'groups': reg_data['neighborhood_code']})

In [23]:
# how does it look?
results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          v_noinknovote   R-squared:                       0.198
Model:                            WLS   Adj. R-squared:                  0.191
Method:                 Least Squares   F-statistic:                     28.14
Date:                Thu, 18 Jan 2018   Prob (F-statistic):           8.37e-25
Time:                        09:24:42   Log-Likelihood:                -1422.2
No. Observations:                2304   AIC:                             2886.
Df Residuals:                    2283   BIC:                             3007.
Df Model:                          20                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1569      0.161      0.976      0.333      -0.164       0.478
tv_access[T.yes]      0.0701      0.030      2.323      0.023       0.010       0.130
advice_pir[T.yes]    -0.0909      0.026     -3.460      0.001      -0.143      -0.038
t1_nocc               0.0877      0.069      1.273      0.207      -0.050       0.225
t2_nocc               0.1252      0.070      1.780      0.080      -0.015       0.266
NAregist_vot_F        0.0006      0.000      4.700      0.000       0.000       0.001
hhsize               -0.0006      0.002     -0.347      0.730      -0.004       0.003
age                   0.0025      0.001      3.306      0.002       0.001       0.004
nc05                 -0.0095      0.009     -1.029      0.307      -0.028       0.009
nall                  0.1983      0.045      4.407      0.000       0.108       0.288
mobil_ins_allow       0.0611      0.019      3.201      0.002       0.023       0.099
d_hhsize              0.0054      0.101      0.053      0.958      -0.196       0.207
d_age                 0.0959      0.110      0.875      0.385      -0.123       0.315
village_code_2       -0.3473      0.113     -3.068      0.003      -0.573      -0.121
village_code_3       -0.5972      0.124     -4.826      0.000      -0.844      -0.350
village_code_4       -0.3129      0.131     -2.393      0.020      -0.574      -0.052
village_code_5       -0.4217      0.150     -2.815      0.006      -0.721      -0.123
village_code_6        0.0504      0.117      0.432      0.667      -0.183       0.283
village_code_9       -0.3897      0.156     -2.499      0.015      -0.701      -0.078
village_code_10      -0.2229      0.140     -1.592      0.116      -0.502       0.057
village_code_11      -0.3092      0.125     -2.478      0.016      -0.558      -0.060
==============================================================================
Omnibus:                      419.327   Durbin-Watson:                   1.605
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              138.241
Skew:                          -0.376   Prob(JB):                     9.58e-31
Kurtosis:                       2.065   Cond. No.                     7.66e+03
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 7.66e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Comparing this second result to the first regression result, you can see that only the values of d_age have changed. However, the p-value still indicates that this variable is of little statistical significance.

### Part V: Wald Tests

In addition to the main regession model, Giné and Mansuri test the similarity of the two main variables of interest, t1_nocc and t2_nocc.

In [24]:
# test to determine the probability that the effects of the two treatments are equal
test_equal = results2.wald_test('t1_nocc - t2_nocc = 0')
test_equal

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[0.58235484]]), p=0.4481099262006929, df_denom=66, df_num=1>

In [25]:
# test to determine the probability that the two treatments are jointly significant
test_joint_sig = results2.wald_test('t1_nocc, t2_nocc')
test_joint_sig

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[1.58943237]]), p=0.21175157900161948, df_denom=66, df_num=2>

### Part VI: Saving the results to a .csv file
Since I am not aware of any Python package available for import that reproduces the results of Stata's custom outreg2 command, we can define a custom function below that takes in regression results, plus any number of additional data, and saves it all as a .csv file. The function is not quite as robust as outreg2, in terms of being able to append additional columns and formatting, but it gets the job done.

In [26]:
def py_outreg2(reg_results, coeffs: list, full_filepath: str, round_num=3, **kwargs):
    csv = open(full_filepath, 'w') # open .csv at specified location
    
    # write the coefficient value and standard error for all specified coefficients
    for coeff in coeffs:
        csv.write(str(coeff) + ',' + str(round(reg_results.params[coeff],round_num)) + '\n')
        csv.write(             ',' + str(round(reg_results.bse[coeff],round_num))    + '\n')
    
    csv.write('\n') # separate coefficients from other regression values
    csv.write('R-squared,'    + str(round(reg_results.rsquared,round_num)) + '\n') # write the R^2 value of the regression
    csv.write('Observations,' + str(reg_results.nobs)     + '\n') # write the number of observations
    csv.write('\n') # separate other regression values from additional test statistics
    
    # write any additional test statistics specified by the user
    for key in kwargs:
        csv.write(key + ',' + str(round(float(kwargs[key]),round_num)) + '\n')
        
    print('File created at: ' + full_filepath) # confirm file creation and location

In [27]:
py_outreg2(results2, ['t1_nocc', 't2_nocc'], '/Users/adouglas/Desktop/test.csv',
           test_equal=test_equal.pvalue, test_joint_sig=test_joint_sig.pvalue)

File created at: /Users/adouglas/Desktop/test.csv
